## Compare Graph Signatures

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 

from pygsp import graphs, filters, plotting, reduction
import os.path
from global_graph import GlobalGraph
from wavelet_evaluator import WaveletEvaluator


%load_ext autoreload
%autoreload 2

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

print(f'\n\nLoading....')

dataroot = '/mnt/data/datasets/fgsp/'
datasets = ['test2/']

n_datasets = len(datasets)
opt_signals = []
est_signals = []
opt_trajectory = []
est_trajectory = []
est_raw_trajectory = []
opt_graph_coords = []
opt_graph_adj = []
for ds in datasets:
    ds_path = dataroot + ds
    print(f'Loading data from {ds_path}')
    opt_signal_path = ds_path + 'opt_signal.npy'
    opt_traj_path = ds_path + 'opt_trajectory.npy'
    est_signal_path = ds_path + 'est_signal.npy'
    est_traj_path = ds_path + 'est_trajectory.npy'  
    est_raw_traj_path = ds_path + 'est_raw_trajectory.npy'  
    opt_graph_coords_path = ds_path + 'opt_graph_coords.npy'
    opt_graph_adj_path = ds_path + 'opt_graph_adj.npy'
 
    if os.path.isfile(opt_signal_path) and os.path.isfile(est_signal_path):
        opt_signals.append(np.load(opt_signal_path))
        est_signals.append(np.load(est_signal_path))
    else:
        print(f'Signal files are not available:')
        print(f'Opt file: {opt_signal_path}')
        print(f'Est file: {est_signal_path}')
        
    if os.path.isfile(opt_traj_path) and os.path.isfile(est_traj_path) and os.path.isfile(est_raw_traj_path):
        opt_trajectory.append(np.load(opt_traj_path))
        est_trajectory.append(np.load(est_traj_path))
        est_raw_trajectory.append(np.load(est_raw_traj_path))
    else:
        print(f'Trajectory files are not available.')
        print(f'Opt file: {opt_traj_path}')
        print(f'Est file: {est_traj_path}')
        print(f'Est raw file: {est_raw_traj_path}')
    
    if os.path.isfile(opt_graph_coords_path) and os.path.isfile(opt_graph_adj_path):
        opt_graph_coords.append(np.load(opt_graph_coords_path))
        opt_graph_adj.append(np.load(opt_graph_adj_path))
    else:
        print(f'Graph files are not available.')
        print(f'Coords file: {opt_graph_coords_path}')
        print(f'Adj file: {opt_graph_adj_path}')
    
print(f'=======================================')



Loading....
Loading data from /mnt/data/datasets/fgsp/test2/


In [36]:
global_graph_per_dataset = []
robot_graph_per_dataset = []

for i in range(0, n_datasets):
    global_graph = GlobalGraph()
    global_graph.build_graph_from_coords_and_adj(opt_graph_coords[i], opt_graph_adj[i])    
    global_graph_per_dataset.append(global_graph)
    
    robot_graph = GlobalGraph()        
    robot_graph.build_from_poses(est_trajectory[i][:,1:4])    
    robot_graph_per_dataset.append(robot_graph)

n_graphs = len(global_graph_per_dataset)
assert n_graphs  == len(robot_graph_per_dataset)
print(f'Built {n_graphs} global and robot graphs.')

global_graph_per_dataset[0].G.plot()
robot_graph_per_dataset[0].G.plot()

Built 1 global and robot graphs.


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:941: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

(<Figure size 1000x500 with 1 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f33d3a9ee48>)

In [15]:
i = 0
opt_origin = opt_trajectory[i][0,1:4]
est_origin = est_trajectory[i][0,1:4]

opt_pos_signal = (opt_trajectory[i][:,1:4] - opt_origin).squeeze()
est_pos_signal = (est_trajectory[i][:,1:4] - est_origin).squeeze()

x_opt = np.linalg.norm(opt_pos_signal, ord=2, axis=1)
x_est = np.linalg.norm(est_pos_signal, ord=2, axis=1)

print(f'Global graph[{i}] opt dirichlet energy is {global_graph_per_dataset[0].G.dirichlet_energy(x_opt)}')
print(f'Global graph[{i}] est dirichlet energy is {global_graph_per_dataset[0].G.dirichlet_energy(x_est)}')
print(f'diff is {global_graph_per_dataset[0].G.dirichlet_energy(x_opt) - global_graph_per_dataset[0].G.dirichlet_energy(x_est)}')

Global graph[0] opt dirichlet energy is 4114.66470153841
Global graph[0] est dirichlet energy is 4303.282093437623
diff is -188.61739189921354


In [4]:
fig, ax = plt.subplots(1, 2)
plt.subplot(1, 2, 1)
global_adj_plot = plt.imshow(global_graph_per_dataset[0].G.W.toarray())
plt.subplot(1, 2, 2)
robot_adj_plot = plt.imshow(robot_graph_per_dataset[0].G.W.toarray())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [30]:
i = 0
global_graph_eval = WaveletEvaluator(7)
robot_graph_eval = WaveletEvaluator(7)
global_graph_eval.compute_wavelets(global_graph_per_dataset[i].G)
robot_graph_eval.compute_wavelets(robot_graph_per_dataset[i].G)

x_opt = opt_signals[i]
x_est = est_signals[i]
global_psi = global_graph_eval.get_wavelets()
robot_psi = robot_graph_eval.get_wavelets()
W_opt = global_graph_eval.compute_wavelet_coeffs(x_opt)
W_est = robot_graph_eval.compute_wavelet_coeffs(x_est)
assert W_opt.shape[0] == W_est.shape[0]
assert W_opt.shape[1] == W_est.shape[1]

In [11]:
steps = [x*20 for x in range(50)]
n_nodes = W_opt.shape[0]

global_data = []
robot_data = []
for i in range(0, n_nodes):    
    global_wavelet_coefficients = [np.mean(np.exp(W_opt[i,:]*1.0*step*1j)) for step in steps]
    robot_wavelet_coefficients = [np.mean(np.exp(W_est[i,:]*1.0*step*1j)) for step in steps]
    global_data.append(global_wavelet_coefficients)
    robot_data.append(robot_wavelet_coefficients)

In [31]:
fig, ax = plt.subplots(1, 5)
plt.subplot(1, 5, 1)
# global_energy = np.abs(np.array(global_data))
# robot_energy = np.abs(np.array(robot_data))

global_energy = np.abs(np.array(W_opt))
robot_energy = np.abs(np.array(W_est))

plt.imshow(global_energy)
plt.subplot(1, 5, 2)
plt.imshow(robot_energy)

difference_low = np.linalg.norm(global_energy[:,0:2] - robot_energy[:,0:2], axis=1)
difference_low = np.reshape(difference_low, (-1,1))
# difference_low = difference_low / np.max(difference_low)
plt.subplot(1, 5, 3)
diff_plot = plt.imshow(np.reshape(np.repeat(difference_low, 50, axis=0), (-1,50)))
plt.colorbar(diff_plot)

difference_mid = np.linalg.norm(global_energy[:,2:5] - robot_energy[:,2:5], axis=1)
difference_mid = np.reshape(difference_mid, (-1,1))
# difference_mid = difference_mid / np.max(difference_mid)
plt.subplot(1, 5, 4)
diff_plot2 = plt.imshow(np.reshape(np.repeat(difference_mid, 50, axis=0), (-1,50)))
plt.colorbar(diff_plot2)

difference_high = np.linalg.norm(global_energy[:,5:8] - robot_energy[:,5:8], axis=1)
# difference_high = difference_high / np.max(difference_high)
difference_high = np.reshape(difference_high, (-1,1))
plt.subplot(1, 5, 5)
diff_plot3 = plt.imshow(np.reshape(np.repeat(difference_high, 50, axis=0), (-1,50)))
plt.colorbar(diff_plot3)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [34]:
fig, axes = plt.subplots(7, 1, figsize=(8, 8))
#fig.subplots_adjust(hspace=0.5)

for i in range(0, 7):
    axes[i].plot(W_opt[:,i], label='Optimized')
    axes[i].plot(W_est[:,i], label='Estimated')    
    axes[i].legend()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [35]:
top_n = 20

ind = np.unravel_index(np.argsort(difference_low, axis=None), difference_low.shape)
peaks = ind[0][-top_n:][::-1]
_ = global_graph_per_dataset[0].G.plot(title=f'Top {top_n} low difference', highlight=peaks)

ind = np.unravel_index(np.argsort(difference_mid, axis=None), difference_mid.shape)
peaks = ind[0][-top_n:][::-1]
_ = global_graph_per_dataset[0].G.plot(title=f'Top {top_n} mid difference', highlight=peaks)

ind = np.unravel_index(np.argsort(difference_high, axis=None), difference_high.shape)
peaks = ind[0][-top_n:][::-1]
_ = global_graph_per_dataset[0].G.plot(title=f'Top {top_n} high difference', highlight=peaks)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Baseline Solution (Simple distance-based)

In [38]:
def compare_trajectories(est_traj, opt_traj):    
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(est_traj[:,1], est_traj[:,2], est_traj[:,3], label='Estimated Trajectory')
    ax.scatter(opt_traj[:,1], opt_traj[:,2], opt_traj[:,3], label='Optimized Trajectory')
    plt.legend()
    plt.show()
    
i = 0
print(f'Synced signal shapes are {est_trajectory[i].shape} and {opt_trajectory[i].shape}')
compare_trajectories(est_trajectory[i], opt_trajectory[i])

Synced signal shapes are (386, 8) and (386, 8)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [37]:
top_n = 20
diff_simple = np.linalg.norm(est_trajectory[i][:,1:4] - opt_trajectory[i][:,1:4], axis=1)
ind = np.unravel_index(np.argsort(diff_simple, axis=None), diff_simple.shape)
peaks = ind[0][-top_n:][::-1]
_ = global_graph_per_dataset[0].G.plot(title='Top 20 difference L2', highlight=peaks)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…